In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from teste_datas import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot

from bokeh import palettes

In [2]:
# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv

In [3]:
fn2 = '../Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'

In [4]:
spec2 = Spec()
if spec2.readSp( fn2 ) >= 0:
    print (spec2.spLngData)
    
counts = spec2.spCounts
print(len(counts))

ret = cwt_complete_analysis(counts)
# ret = gaussian_complete_analysis(counts)

(xs, y0s, ar1, lblimag1, num_feat_imag1, ridges, xpks, ypks,
 ind_ypks, retang_indicess, peaks_masks, cwtpks, nscpks) = ret

4096
4096


In [5]:
# PAREI AQUI: 2020-04-25 Já mudei de ideia... Agora ver a análise com arquivo do CR (I-131).

In [6]:
# Desativado:

In [7]:
(nscpk_ma, cwtpk_ma, pospk_ma) = peaks_masks
valid_peaks = ~(cwtpk_ma.mask | nscpk_ma.mask | ~pospk_ma)

In [8]:
default_plot_width, default_plot_height = set_screen_size_by_machine()

# Bokeh graph w/ peaks in all scales

# Gráfico de ridges com qualidade dos picos pelas cores e tamanhos:

# Navy 2:     pontos dos ridgest
# Vermelho 8: picos válidos
# Laranja 8:  inválido, aparece em poucas escalas  
# Verde 6:    inválido, muito pequeno (pelo coeficiente cwt)
# Ciano 3:    inválido, forma ruim (cwt perto dos extremos do ridge)

bokridges1 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   # x_axis_label='channels',
   y_axis_label='scale'
)
bokridges1.circle(ar1[:,0], ar1[:,1], color="navy",size=2)
bokridges1.circle(xpks[valid_peaks], ypks[valid_peaks], color="red",size=8)
bokridges1.circle(xpks[nscpk_ma.mask], ypks[nscpk_ma.mask], color="orange",size=8)
bokridges1.circle(xpks[cwtpk_ma.mask], ypks[cwtpk_ma.mask], color="green",size=6)
bokridges1.circle(xpks[~pospk_ma], ypks[~pospk_ma], color="cyan",size=3)
bokridges2 = figure(
   plot_width =default_plot_width,
   plot_height=int(default_plot_height*0.5),
   tools="pan,box_zoom,reset,save",
   y_axis_type="log", y_range=[1.0e1, 1.0e5],
   x_axis_label='channels', y_axis_label='counts'
)
bokridges2.line(xs, y0s, color="green")
# https://docs.bokeh.org/en/latest/docs/reference/layouts.html
p = gridplot([bokridges1, bokridges2], ncols=1)
show(p)

In [9]:
valid_xpks = np.asarray(xpks)[valid_peaks]
valid_xpks

array([ 415,  814, 1120])

In [10]:
valid_ypks = np.asarray(ypks)[valid_peaks]
valid_ypks

array([ 2.62987755,  2.41991837, 10.39836735])

In [11]:
valid_cwtpks = np.asarray(cwtpks)[valid_peaks]
valid_cwtpks

array([1.37810486e+06, 1.59150904e+04, 3.55332542e+01])

In [12]:
xs

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 4.093e+03, 4.094e+03,
       4.095e+03])

In [13]:
y0s

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
step2 = cwt_calc_peaks(valid_xpks, valid_ypks, valid_cwtpks, xs, y0s)

In [15]:
xpks[pospk_ma]


array([ 415,  814, 1013, 1120, 1211, 1215, 1257, 1313, 1473, 1636, 1731,
       1859, 1869, 2017, 2080, 2181, 2196, 2218, 2288, 2322, 2363, 2419,
       2627, 2671, 2704, 2710, 2795, 2820, 2919, 3065, 3087, 3277, 3298,
       3345, 3381, 3509, 3588, 3623, 3633, 3741, 3768, 3820, 3857, 3933,
       3965, 3978, 1605, 1659, 2144, 3443, 1149, 2036, 2248, 1527,  983,
       1426, 2739, 3197, 1397, 2529, 1799, 2654, 2559, 3628, 3540, 2052])

In [16]:
(azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
           baseline_spectrum, bl_median_smoother) = step2

In [17]:
azes

[array([410., 411., 412., 413., 414., 415., 416., 417., 418., 419., 420.]),
 array([809., 810., 811., 812., 813., 814., 815., 816., 817., 818., 819.]),
 array([1115., 1116., 1117., 1118., 1119., 1120., 1121., 1122., 1123.,
        1124., 1125.])]

In [18]:
len(azes)

3

In [19]:
bezes

[array([ 0.000000e+00,  4.459400e+03,  1.472880e+04,  9.475620e+04,
         6.779246e+05,  1.410610e+06,  8.039214e+05,  1.302548e+05,
         3.880200e+03, -1.261400e+03,  0.000000e+00]),
 array([    0. ,    37.4,   268.8,  2233.2,  9698.6, 14793. ,  7990.4,
         1438.8,    25.2,   -27.4,     0. ]),
 array([ 0. ,  1.5, -4. ,  0.5, -4. ,  4.5,  7. ,  7.5, 18. , 15.5,  0. ])]

In [20]:
centrpk

[415.0450948212866, 813.8824126392014, 1123.1612903225807]

In [21]:
#azes, bezes, centrpk, ypkregs, fwhms, gau_areas, peak_constructed_spectrum,
#           baseline_spectrum, bl_median_smoother) = step2

In [22]:
bezes[0]

array([ 0.000000e+00,  4.459400e+03,  1.472880e+04,  9.475620e+04,
        6.779246e+05,  1.410610e+06,  8.039214e+05,  1.302548e+05,
        3.880200e+03, -1.261400e+03,  0.000000e+00])

In [23]:
gau_areas

[3387193.4126061485, 37523.162750015166, 173.66309194141255]

In [24]:
# 2020-02-24
# Tudo em progresso! Agora:
# - Fazer os gr'aficos
# - fazer em v'arias passadas o c'alculo at'e n~ao haver picos na linha-base.

In [25]:
# Bokeh graph w/ peaks in all scales
bokpksallscl = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    y_range=[10**1, 10**5], title="real spec",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
bokpksallscl.line(xs, y0s, color="green")
bokpksallscl.line(xs, peak_constructed_spectrum, color="red")
bokpksallscl.line(xs, bl_median_smoother, color="cyan")
bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bokpksallscl)

In [26]:
# Bokeh graph w/ peaks in all scales
bok2 = figure(
   plot_width =default_plot_width,
   plot_height=default_plot_height,
   tools="pan,box_zoom,reset,save",
   # y_axis_type="log",
   # y_range=[10**1, 10**5], title="real spec",
   # y_range=[-1000.0, 1000.0], title="synth",
   x_axis_label='channels', y_axis_label='counts'
)
# bokpksallscl.circle(ar1[:,0], ar1[:,1], legend_label="cwt peaks", color="navy",size=2)
# bokpksallscl.line(xs, y0s, color="green")
bok2.line(xs, peak_constructed_spectrum, color="red")
# bokpksallscl.line(xs, bl_median_smoother, color="cyan")
# bokpksallscl.line(xs, peak_constructed_spectrum+bl_median_smoother, color="blue")

# bokpksallscl.cross(pk_per_scal[ 0], y0s[pk_per_scal[ 0]]*1.01, color="red",size=2)
show(bok2)

In [27]:
# 2020-02-25 Segunda passada

In [28]:
ret2 = cwt_complete_analysis(bl_median_smoother)

(xs, y0s, ar1, lblimag1, num_feat_imag1, ridges, xpks, ypks,
 ind_ypks, retang_indicess, peaks_masks, cwtpks, nscpks) = ret2

In [29]:
(nscpk_ma, cwtpk_ma, pospk_ma) = peaks_masks
valid_peaks = ~(cwtpk_ma.mask | nscpk_ma.mask | ~pospk_ma)